In [1]:
import pandas as pd
import numpy as np

#Legend of variables
#cs = code stress results grid
#gs = general stress results grid

#reads the data as a .csv file. 
#Copy the entire code stress results grid, this function will clean up the columns automatically
#input the csv file here. Ensure the python script file and the csv file are in the same folder.
cs = pd.read_csv('100cs.csv', low_memory=False).tail(-1)
gs = pd.read_csv('100G.csv', low_memory=False).tail(-1)

#deletes the unused columns
#If any of these columns are needed just delete from this array
cs = cs.drop(columns=["Internal1", "Internal2", "PointOrder", 'CombinationOrder', 'PlusSideOrder', 'PercSoilOrder', 'CategoryOrder', 'Equation', 'SIF-In', 'SIF-Out', 'SectMod1', 'Torsion', 'Moment-In', 'Moment-Out', 'AxialForce', 'AxialStress', 'Bending', 'ShearStress', 'Pressure', 'Bending'])

In [2]:
#Data clean up
#Removes all duplicates based on soil points and bend midpoints
points = cs['Point'].str.split(' ').str[0]
points = points.str.split('N').str[0]
points = points.str.split('F').str[0]
points = points.str.split('M').str[0]
pointsg = gs['Point'].str.split(' ').str[0]
pointsg = pointsg.str.split('N').str[0]
pointsg = pointsg.str.split('F').str[0]
pointsg = pointsg.str.split('M').str[0]
gs['Point'] = pointsg
cs['Point'] = points
gs['Total Stress'] = gs['Total Stress'].astype(float)

#Manually calculate 'ratio' column to get more significant digits than the code stress tab gives
cs['Ratio'] = (cs['Stress'].astype(float) / cs['Allowable'].astype(float))*100
cs['Ratio'] = cs['Ratio'].round(2)

In [6]:
#Remove all duplicates from the general stress, sort all points by max total stress
gsSorted = gs.sort_values('Total Stress', ascending=False).drop_duplicates('Point').sort_index()

#Extract only the allowables column from code stress. To be used later to calculate General stress ratio
allowables = cs.sort_values('Allowable', ascending=False).drop_duplicates('Point').sort_index()

#Get general stress in terms of a ratio
merged = pd.merge(allowables, gsSorted, on="Point")
merged['General'] = (merged['Total Stress'].astype(float) / merged['Allowable'].astype(float))*100
merged['General'] = merged['General'].round(2)

#Creates the main code stress results table using a pivot table. 
#"points" are a list of all nodes after cleaning up soil points. Contains many duplicates
#points must be the same size as the data, don't delete non-duplicate nodes
#values= are the stress ratios 'Ratio'
#columns are each nodes
#Index are the rows - stress categories (Expansion, Tresca etc..)
#aggfunc=np.max makes sure to take the maximum at each indexm (node)
table = pd.pivot_table(cs, values='Ratio',index=points, columns='Category', aggfunc=np.max)

#combine the general stresses and code stresses & cleanup the index column
df_merged = table.merge(merged[['Point', 'General']], on='Point')
df_merged = df_merged.set_index('Point').T

Point        A00    A01   A110   A111   A112   A113   A114   A115    A116  \
Expansion  32.76  26.94  38.47  35.99  39.04  37.33  37.63  36.04   38.82   
Hoop       74.97  54.02  90.26  90.26  90.26  90.26  90.26  90.26  100.15   
Occasion   27.39  23.60  32.84  29.26  31.29  29.22  29.39  28.84   38.49   
Sustain    26.98  22.78  32.80  29.20  31.27  29.15  29.30  28.70   38.30   
Tensile 1  73.83  53.92  84.75  85.74  85.41  86.51  87.02  85.83   91.11   
Tensile 2  69.26  50.48  79.87  77.16  80.47  77.88  78.34  77.65   85.90   
General    65.06  47.24  76.25  74.80  76.66  75.21  75.42  75.01   83.09   

Point        A119  ...   WW21   WW22   WW23   WW24   WW25   WW26   WW27  \
Expansion   36.33  ...  16.96  45.56  40.93  33.85  35.73  35.26  42.13   
Hoop       100.15  ...  90.26  90.26  74.97  74.97  74.97  74.97  74.97   
Occasion    33.54  ...  37.82  45.19  39.11  28.30  26.65  26.39  31.15   
Sustain     33.48  ...  37.40  44.61  38.67  27.22  26.57  25.97  30.82   
Tensile 

In [4]:
#Inputs 
#Input all the bend node numbers into the array "BendNodes"
#Figure out this part
BendNodes = df_merged[['A111', 'A112', 'A113', 'A114']]
BendNodes

Point,A111,A112,A113,A114
Expansion,35.99,39.04,37.33,37.63
Hoop,90.26,90.26,90.26,90.26
Occasion,29.26,31.29,29.22,29.39
Sustain,29.20,31.27,29.15,29.30
Tensile 1,85.74,85.41,86.51,87.02
Tensile 2,77.16,80.47,77.88,78.34
General,74.80,76.66,75.21,75.42


In [5]:
#Outputs results to excel
BendNodes.to_excel("output.xlsx")  